In [3]:
import pandas as pd

df = pd.read_csv("/home/username/open_clip/finetuning/train12.csv")
df2 = pd.read_csv("/home/username/open_clip/finetuning/val12.csv")

In [95]:
va_images=df[df["image_path"]>="/scratch/username/data/va_images/"]
britmus_images=df[df["image_path"]<"/scratch/username/data/va_images/"]

In [34]:
# britmus_images.to_csv("/home/username/open_clip/finetuning/val_britmus.csv", index=False)  
# va_images.to_csv("/home/username/open_clip/finetuning/val_va.csv", index=False)  

## analyse keys

In [1]:
import pandas as pd

val6 = pd.read_csv('/home/username/open_clip/finetuning/val6.csv')
print(val6.columns) 

Index(['my_code', 'image_code', 'image_path', 'caption', 'Museum number',
       'Subjects', 'Curators Comments', 'Inscription', 'Technique',
       'Materials', 'Production place', 'Production date', 'Assoc name',
       'Culture', 'Object type', 'Producer name', 'School/style', 'Title'],
      dtype='object')


In [2]:
old_categories=['Subjects', 'Inscription', 'Technique', 'Materials', 'Production place', 'Production date', 
                 'Assoc name', 'Culture', 'Object type', 'Producer name']
new_categories=['systemNumber', 'accessionNumber', 'objectType', 'titles', 'summaryDescription', 
                 'physicalDescription', 'artistMakerPerson', 'artistMakerOrganisations', 'artistMakerPeople', 
                 'materials', 'techniques', 'materialsAndTechniques', 'categories', 'styles', 'collectionCode',
                   'images', 'imageResolution', 'galleryLocations', 'partTypes', 'contentWarnings', 'placesOfOrigin',
                     'productionDates', 'associatedObjects', 'creditLine', 'dimensions', 'dimensionsNote', 'marksAndInscriptions', 
                     'objectHistory', 'historicalContext', 'briefDescription', 'bibliographicReferences', 'production', 'productionType', 
                     'contentDescription', 'contentPlaces', 'associatedPlaces', 'contentPerson', 'associatedPerson', 'contentOrganisations', 
                     'associatedOrganisations', 'contentPeople', 'associatedPeople', 'contentEvents', 'associatedEvents', 'contentOthers', 
                     'contentConcepts', 'contentLiteraryRefs', 'galleryLabels', 'partNumbers', 'accessionNumberNum', 'accessionNumberPrefix', 
                     'accessionYear', 'otherNumbers', 'copyNumber', 'aspects', 'assets', 'recordModificationDate', 'recordCreationDate']
without=['systemNumber',"collectionCode", 'productionType','bibliographicReferences','dimensions','dimensionsNote','creditLine',"images",
          "imageResolution","galleryLocations",'artistMakerPeople','artistMakerOrganisations','accessionNumber',"contentWarnings", 
          "associatedObjects",'contentDescription', 'contentPlaces', 'associatedPlaces', 'contentPerson', 
 'associatedPerson', 'contentOrganisations', 'associatedOrganisations', 'contentPeople', 
 'associatedPeople', 'contentEvents', 'associatedEvents', 'contentOthers', 'contentConcepts', 
 'contentLiteraryRefs', 'galleryLabels', 'partNumbers', 'accessionNumberNum', 'accessionNumberPrefix', 
 'accessionYear', 'otherNumbers', 'number','copyNumber', 'aspects',
 'assets','recordModificationDate','recordCreationDate']
common=['inscription', 'technique', 'materials', 'production']
new_categories_kept=['objectType','titles','summaryDescription','physicalDescription','artistMakerPerson','materials',
'techniques','materialsAndTechniques','categories','styles','partTypes','placesOfOrigin','productionDates','marksAndInscriptions',
'objectHistory','historicalContext','briefDescription','production',]

In [3]:
new_categories=old_categories

In [4]:
print(new_categories_kept)
print(old_categories)

['objectType', 'titles', 'summaryDescription', 'physicalDescription', 'artistMakerPerson', 'materials', 'techniques', 'materialsAndTechniques', 'categories', 'styles', 'partTypes', 'placesOfOrigin', 'productionDates', 'marksAndInscriptions', 'objectHistory', 'historicalContext', 'briefDescription', 'production']
['Subjects', 'Inscription', 'Technique', 'Materials', 'Production place', 'Production date', 'Assoc name', 'Culture', 'Object type', 'Producer name']


In [8]:
# classifiable_brit=["materials=multiclass","technique=multiclass","subjects=multiclass"
#                    "production_place=multiclass","production date=period","culture=multiclass","object_type=multiclass"]

# classifiable_va=["materials=?multiclass","techniques=?multicls","categories=multiclass","styles=multiclass",
#                  "artistMakerPerson?","partTypes=multiclass","placesOfOrigin=poss?multi"
#                  "productionDates=period"]
classifiable_va=["materials","techniques","categories","styles"
                 ,"partTypes","productionDates"]
recall_va=["objectType","titles","physical_description","summaryDescription","marksAndInscriptions"
          "objectHistory","historicalContext","briefDescription","production","placesOfOrigin","artistMakerPerson"]
redundant=["materialsAndTechniques"]
classifiable_brit=["Materials","Technique","Subjects","School/style"
                   "Production place","Production date","Culture","Object type"]
recall_brit=["Assoc name","Producer name","Inscription","Curators Comments","Title"]

In [9]:
import json
import pandas as pd

va_path="/home/username/open_clip/finetuning/train6.csv"
brit_df = pd.read_csv(va_path)

In [106]:
va_path="/home/username/open_clip/finetuning/val6.csv"
brit_df2 = pd.read_csv(va_path)

In [114]:
new_brit_df=pd.concat([brit_df, brit_df2], axis=0, ignore_index=True)

In [115]:
brit_df=new_brit_df

In [10]:
import re
def process(text,cls):
    original_text=text
    try:
        if cls=="Production date":
            text=text.lower()

            if "or" in text and "before" not in text and "historic" not in text:
                pos=text.index("or")
                text=text[:pos]

            
            if "pre-historic" in text:
                text="pre-historic"
                set_text=set([text])
                return set_text

            for i in ["summer","winter","autumn","spring","june","july","august","september","october","november","december","january","february","march","april","may","jun","jul","aug","sept","sep","oct","nov","jan","feb","mar","apr"]:
                if i in text:
                    try:
                        val=int(text[-4:])
                        text=text[-4:]
                    except:
                        text="unknown"
                        set_text=set([text])
                        return set_text
            
            for p1,p2 in [("first century","1st century"),("second century","2nd century"),("third century","3rd century"),("forth century","4th century"),("fifth century","5th century"),("sixth century","6th century"),("seventh century","7th century"),("eighth century","8th century"),("ninth century","9th century"),("tenth century","10th century"),("eleventh century","11th century"),("twelfth century","12th century"),("thirteenth century","13th century"),("fourteenth century","14th century"),("fifteenth century","15th century"),("sixteenth century","16th century"),("nineteen","19"),("eighteen","18"),("seventeen","17"),("twentie","20")]:
                            text=text.replace(p1,p2)

            for i in ["perhaps","perhap","copyright","from","between","btwn","after","(chain and pendant)","late ptolemaic", "roman periods","middle","possiby","around","centuy","fourth","earlier","circa","pre-","ormoulu mounts","first", "about","2nd half","lte","mid.","later","probably", "before","second","mid-century","century","early","mid to late","late","last","quarter","half","(?)","ca.","mid","possibly","or","ca","of", "and"]:
                text=text.replace(i,"")
            # print(text)
                
            if original_text=="later ":
                print("text",text)
            if text=="" or text==" ":
                text="unknown"
                set_text=set([text])
                return set_text
           
            # print(text)
            for p1,p2 in [("b.c.","bc"),("bc.","bc"),("b.c","bc"),("c.",""),("bce","bc"),("ce","")]:
                text=text.replace(p1,p2)
            if "bc" not in text:
                text=text.replace("c","")
            else: 
                pos=text.index("c")
                # print("pos",pos)
                if pos!=0 and text[pos-1:pos+1]!="bc":
                    text=text[pos+1:]
                elif pos==0:
                    text=text[pos+1:]
                # print(text)

            for i in ["[","]",".",";",":","\t","&",'"',"(",")","'","#","%","$","@","*","^","=","+","{","}","?","!","~","`","<unauthorised>","<i>","<\i>","<aat>"]:
                text=text.replace(i,"")
            for p1,p2 in [("//","/"),("rd","th"),("nd","th"),("st","th"),(" ",""),(" - ","-"),("th","00"),("ad",""),("ac",""),("s",""),("to","-"),("ce","")]:
                text=text.replace(p1,p2)
            
            if "/" in text:
                if text.count("/")==4:
                    text=text[6:10]+"-"+text[-4:]
                elif text.count("/")==2:
                    elems=text.split("/")
                    # print("elems",elems,elems[1],len(elems[1]))
                    if len(elems[1])==3:
                        # print("text",text)
                        # text=text.replace(" ","")
                        text=elems[2].replace(" ","")+"-"
                    elif "-" not in elems[1]:
                        text=text[6:10]+"-"+text[-4:]
                    else:
                        text=text[3:7]+"-"+text[-4:]
                else:
                    text=text.replace("/","")
                
            if "th" not in text:
                if len(text)==4:
                    text+="-"

            if text[0]=="-":
                new_text="_"+text[1:]
                text=new_text
            # print(text)
            text=text.replace("--","-_")
            # copy_text=text
            # text=text.replace("_","")

            # print("text",text)
            if text.count("-")>1:
                text=text[-4:]

            if text=="" or text==" ":
                text="unknown"
                set_text=set([text])
                return set_text
            

            if "-" in text:
                text=text.replace(" ","")
                elems=text.split("-")
                new_elems=[]
                # print(elems)
                # new_text=""
                if len(elems)==2 and elems[1]!="" and elems[1]!=" ":
                    if len(elems[0])==4 and len(elems[1])<3:
                        # print("elems",elems[1])
                        elems[1]=str(int(elems[0])//100*100+int(elems[1]))
                        # print(elems[0],elems[1])
                for i,e in enumerate(elems):
                    # sign_e=e[0]
                    # print(e)
                    if "bc" in e:
                        e="_"+e.replace(" ","").replace("bc","")
                        elems[i]=e


                    e=e.replace("_","")
                    try:
                        if len(elems)==2:
                            # print(elems,elems[1],"da",e)
                            if e!="":
                                # print("e",e)
                                ie=int(e)
                                # print(ie)
                                if len(e)>1 and len(e)<5:
                                    ie=ie//100+1
                                    #print(ie)
                                    if len(new_elems)>0 and (str(ie)+"th")!=new_elems[0]:
                                        new_elems.append(str(ie)+"th")
                                        # print("aici",ie,new_elems[0])
                                    if len(new_elems)==0:
                                        new_elems.append(str(ie)+"th")
                                        # print("acolo")
                    except:
                        # print("except?", original_text,";", text, elems,new_elems)
                        # print(ie)
                        return set(["unknown"])
                        if e !="":
                            new_elems.append(e)
                for i in range(len(new_elems)):
                    sign_e=elems[i][0]
                    e=new_elems[i]
                    if sign_e=="_":
                        new_elems[i]=new_elems[i].replace("th","thbc")
                    else:
                        new_elems[i]=new_elems[i].replace("th","thac")


                if len(new_elems)>1:
                    text=new_elems[0]+"-"+new_elems[1]
                else:
                    text=new_elems[0]

                for p1,p2 in [("3rd","3th"),("2nd","2th"),("1st","1th")]:
                    text=text.replace(p2,p1)

                if text=="" or text==" ":
                    text="unknown"

                if "ac" not in text and "bc" not in text and text!="pre-historic" and text!="ironage":
                    text="unknown"
                set_text=set([text])
            else:
                if "ac" not in text and "bc" not in text and text!="pre-historic" and text!="ironage":
                    text="unknown"
                set_text=set([text])
            return set_text
        



        else:

            text=text.lower()
            re.sub("\(\w*\)","",text)
            re.sub("<\w*>","",text)
            re.sub("</.*>","",text)
            re.sub("[3-9][0-9][0-9][0-9]","",text)
            
            
            text=text.replace("thearly","th")

            
            for i in ["[","]",".",";",":","\t","&",'"',"(",")","'","#","%","$","@","*","^","=","+","{","}","/","?","!","~","`","<unauthorised>","<i>","<\i>","<aat>"]:
                text=text.replace(i,"")
            text=text.replace("thearly","th")
            re.sub("\(\w*\)","",text)
            re.sub("<\w\w*>","",text)
            re.sub("</.*>","",text)
            re.sub("[3-9][0-9][0-9][0-9]","",text)
            
            set_text=set()
            elems=text.split(" ")
            

            if text=="":
                return set()
            #if " " in text:
            for i in elems:
                if i not in ["and","to","the","of","by","as","a"] and i!="":
                    if i != "-":
                        ok=1
                        for c in range(10):
                            if str(c) in i:
                                for term in ["th","st","nd","rd","D"]:
                                    if term not in i:
                                        if len(i)<4:
                                            ok=0
                        if ok:
                            set_text.add(i)
                            

            # else:
            #     set_text=set([text])
            if not text:
                return set(["unknown"])
            return set_text
    except:
        # print("PROBLEM:", original_text,";",text) 
        set_text=set([original_text])
        # return set_text  
        return set(["unknown"])
        # return 0
    
    

In [116]:
va_dict=brit_df
classifiable_va=classifiable_brit

In [ ]:
print(brit_df.at[0,"image_path"])

In [ ]:
for j,i in enumerate(brit_df["image_path"]):
    print(j,i)
    print(j,brit_df["image_path"][j])
    break
    

In [130]:
print(len(brit_df["image_code"]))

112611


In [119]:
va_dict={}
# print(brit_df.keys())
for i in brit_df["image_code"]:
    va_dict[i]={}
for val in brit_df:
    # print(brit_df[i])
    # print(j,val,i)
    # print("jnul",j,val,brit_df.at[j,val])
    if val!="my_code" and val!="image_code":
        for j,i in enumerate(brit_df[val]):
            va_dict[brit_df.at[j,"image_code"]][val]=str(i)
        # print("here",i,va_dict[i][val])
        # break


In [122]:
va_keys_values={}
va_keys_values2={}
for k in classifiable_va:
    va_keys_values[k]=set()
for k in va_dict:
    for cls in va_dict[k]:
        va_keys_values2[cls]={}
    break
for k in va_dict:
    for cls in va_dict[k]:
        va_keys_values2[cls][k]=set()   

In [123]:
#2869 were too hard to parse - I put them as unknown
all_results={}
all_results_key={}

count=0
for k in va_dict:
    for cls in va_dict[k]:
        all_results[cls]={}
    break
for k in va_dict:
    all_results_key[k]={}
for k in va_dict:
    for cls in va_dict[k]:
        # all_results[cls]=[]
        all_results_key[k][cls]={}
        if cls in classifiable_va:
            elems=va_dict[k][cls].replace("/n","").replace(" ,",",").replace(", ",",").split(",")
            for elem in elems:
                if elem!="":
                    res=process(elem,cls)
                    # if cls=="productionDates":
                    if res!=0:
                        all_results[cls][elem]=res
                        va_keys_values[cls].update(res)
                        va_keys_values2[cls][k].update(res)
                        # all_results_key[k][cls][elem]=res
                    else:
                        count+=1
                elif len(elems)==1:
                    all_results[cls][elem]=set(["unknown"])
                    va_keys_values2[cls][k].update(set(["unknown"]))
                    # all_results_key[k][cls][elem]=set(["unknown"])

        else:
            elems=va_dict[k][cls].replace("/n","").replace(" ,",",").replace(", ",",").split(",")
            for elem in elems:
                if elem!="":
                    all_results[cls][elem]=set([elem])
                    all_results_key[k][cls][elem]=set([elem])
                    va_keys_values2[cls][k].update(set([elem]))
                elif len(elems)==1:
                    all_results[cls][elem]=set(["unknown"])
                    all_results_key[k][cls][elem]=set(["unknown"])
                    va_keys_values2[cls][k].update(set(["unknown"]))
                # va_keys_values[cls][elem]=all_results[cls][elem]
                
        
print(count)

0


## create eval zeroshot csv

In [90]:
import pandas as pd

df1 = pd.read_csv("/home/username/open_clip/finetuning/val12.csv")
df2 = pd.read_csv("/home/username/open_clip/finetuning/train12.csv")

# va_images=df1[df1["image_path"]>="/scratch/username/data/va_images/"]

print(df1.keys())
old_info={"key":[],"image_path":[],"caption":[]}
images_paths=[]
captions=[]
for i in range(len(list(df1["image_path"]))):
    old_info["key"].append(df1["key"][i])
    old_info["image_path"].append(df1["image_path"][i])
    old_info["caption"].append(df1["caption"][i])
for i in range(len(list(df2["image_path"]))):
    old_info["key"].append(df2["key"][i])
    old_info["image_path"].append(df2["image_path"][i])
    old_info["caption"].append(df2["caption"][i])

old_info_df=pd.DataFrame(old_info)
old_info=old_info_df[old_info_df["image_path"]<"/scratch/username/data/va_images/"]
# print("va_image" in df1["image_path"][1])
# print("va_image" in df1['image_path'])
# print(old_info)

Index(['key', 'image_path', 'caption'], dtype='object')


In [131]:
print(len(old_info["key"]))
print(len(va_keys_values2["image_path"]))
print(len(britmus_images["key"]))

88581
106611
5687


In [124]:
json_classes={}
classes_to_id={}

classes_ids={}
for k in all_results_key:
    for cls in all_results_key[k]:
        json_classes[cls]={}
        classes_ids[cls]=0
        classes_to_id[cls]={}
    break


# va_keys_values2[cls][k]
for cls in va_keys_values2:
        for k in va_keys_values2[cls]:
            elem=va_keys_values2[cls][k]
            for e in elem:
                e=str(e).replace("'","").replace('"',"").replace("\\r","").replace("\\n","").replace("\\t","").lower()
                if e not in classes_to_id[cls]:
                    idx=classes_ids[cls]
                    idx+=1
                    json_classes[cls][idx]=e
                    classes_to_id[cls][json_classes[cls][idx]]=idx
                    classes_ids[cls]=idx

        

In [134]:
# import json
# with open("/home/username/open_clip/finetuning/b_classes_new.json","w") as f:
#     json.dump(json_classes,f)

In [126]:
"024289_2007-6001-8039.jpg" in va_keys_values2["image_path"]

True

In [135]:
for cls in va_keys_values2:
    print(cls)

image_path
caption
Museum number
Subjects
Curators Comments
Inscription
Technique
Materials
Production place
Production date
Assoc name
Culture
Object type
Producer name
School/style
Title


In [136]:
#verifica daca all_results si keys sunt in aceeasi ordine
new_df=old_info.copy()
bad=[]
count=0

for cls in va_keys_values2:
    new_list=[]
    if cls == 'image_path' or cls == 'caption': continue
    for k in old_info["key"]:
        list1=[]
        # print(cls,k)
        elem=va_keys_values2[cls][k]
        for e in elem:
            e=str(e).replace("'","").replace('"',"").replace("\\r","").replace("\\n","").replace("\\t","").lower()
            if e in classes_to_id[cls]:
                list1.append(classes_to_id[cls][e])
            else:
                print(e,k,cls,elem)
                bad.append(e)
                exit(0)
        new_list.append(list1)
    new_df[cls]=new_list
        


print(len(bad))

0


In [ ]:
new_info_df=pd.DataFrame()
new_info=old_info_df[new_df["image_path"]<"/scratch/username/data/va_images/"]

In [138]:
# new_df.to_csv('/home/username/open_clip/finetuning/train_brit_classes_new.csv',index=False)

In [ ]:
# new_df.to_csv('/home/username/open_clip/finetuning/val_brit_classes_new.csv',index=False)

### get britmus

In [1]:
import json
import pandas as pd

va_path='/home/username/open_clip/finetuning/all_brit_classes_new.csv'
brit_df = pd.read_csv(va_path)

In [4]:
britmus_train=brit_df[brit_df["key"].isin(df["key"])]
britmus_val=brit_df[brit_df["key"].isin(df2["key"])]

In [8]:
print(britmus_val["key"].keys())
print(britmus_train["key"].keys())

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       5677, 5678, 5679, 5680, 5681, 5682, 5683, 5684, 5685, 5686],
      dtype='int64', length=5687)
Index([ 5687,  5688,  5689,  5690,  5691,  5692,  5693,  5694,  5695,  5696,
       ...
       88571, 88572, 88573, 88574, 88575, 88576, 88577, 88578, 88579, 88580],
      dtype='int64', length=82894)


In [11]:
# britmus_train.to_csv('/home/username/open_clip/finetuning/train_brit_dataset_new.csv',index=False)
# britmus_val.to_csv('/home/username/open_clip/finetuning/val_brit_dataset_new.csv',index=False)